## Analysis of Texas Court Data

The code below analyzes court data from the Texas Office of Court Administration. For more details and context, please read [this page](https://github.com/BuzzFeedNews/2016-01-texas-municipal-court-data).

In [1]:
import pandas as pd

In [2]:
base_path = "../data/"

In [3]:
outcome_files = [
    "Municipal_Courts-MUN_Municipal_Court_Add_Activity_ByCity_N_FY15.xls",
    "Municipal_Courts-MUN_Municipal_Court_Add_Activity_ByCity_N_FY14.xls",
    "Municipal_Courts-MUN_Municipal_Court_Add_Activity_ByCity_N_FY13.xls"
]

In [4]:
case_number_files = [
    "Municipal_Courts-MUN_Municipal_Court_Activity_ByCity_N_FY15.xls",
    "Municipal_Courts-MUN_Municipal_Court_Activity_ByCity_N_FY14.xls",
    "Municipal_Courts-MUN_Municipal_Court_Activity_ByCity_N_FY13.xls",
]

In [5]:
def load_outcomes(file_path):
    data = pd.read_excel(base_path + file_path, header=None, skiprows=12)
    return data

In [6]:
muni_case_outcomes_raw = pd.concat([ load_outcomes(f) for f in outcome_files ])

In [7]:
def load_case_numbers(file_path):
    data = pd.read_excel(base_path + file_path, header=None, skiprows=15)
    return data

In [8]:
case_numbers_raw = pd.concat([ load_case_numbers(f) for f in case_number_files ])

In [9]:
def is_city_name(city_name):
    if pd.isnull(city_name): return False
    if city_name.strip() == "City": return False
    if "TOTALS" in city_name: return False
    if "Run Report" in city_name: return False
    return True

In [10]:
def clean_up_outcomes(df):
    filtered = df[
        df[3].apply(is_city_name)
    ]
    return pd.DataFrame({
        "city": filtered[3],
        "jail_credit": pd.to_numeric(filtered[31])
    })

In [11]:
def clean_up_case_numbers(df):
    filtered = df[
        df[1].apply(is_city_name)
    ]
    return pd.DataFrame({
        "city": filtered[1],
        "population": filtered[2],
        "criminal_case_disposed": pd.to_numeric(filtered[25])
    })

In [12]:
muni_case_outcomes = clean_up_outcomes(muni_case_outcomes_raw)
muni_case_outcomes.head()

,city,jail_credit
90,Abernathy,12
91,Abilene,4105
92,Addison,527
93,Alamo,61
94,Alamo Heights,137


In [13]:
outcomes_3y = pd.DataFrame(muni_case_outcomes.groupby("city")["jail_credit"].sum())
outcomes_3y.head()

,jail_credit
city,
Abernathy,48
Abilene,13271
Addison,2185
Alamo,316
Alamo Heights,410


In [14]:
case_numbers = clean_up_case_numbers(case_numbers_raw)
case_numbers.head()

,city,criminal_case_disposed,population
0,Abernathy,287,2750
2,Abilene,17391,120958
3,Addison,7599,15457
4,Alamo,4374,19224
5,Alamo Heights,5394,7806


In [15]:
grp = case_numbers.groupby("city")
case_numbers_3y = pd.DataFrame({
    "criminal_case_disposed": grp["criminal_case_disposed"].sum(),
    "population": grp["population"].first()
})
case_numbers_3y.head()

,criminal_case_disposed,population
city,,
Abernathy,749,2750
Abilene,55740,120958
Addison,27775,15457
Alamo,12251,19224
Alamo Heights,16077,7806


In [16]:
cities = outcomes_3y.join(case_numbers_3y, how="outer")

In [17]:
print("{0:d} cities are included in the municipal court data.".format(len(cities)))

962 cities are included in the municipal court data.


According to the Office of Court Administration, zeros can mean either:

    1) The court submitted reports but may not have reported information for that category or
    2) The court reported zero for that item.
    
Because it is unclear, BuzzFeed did not include these courts in its ranking.

In [18]:
cities_with_data = cities[cities["jail_credit"] != 0].\
    dropna(subset=["criminal_case_disposed", "jail_credit"])


In [19]:
print("""Of those, {0} cities' data includes both the number of 
criminal cases disposed and the number of cases satisfied by jail credit.
""".format(len(cities_with_data)))

Of those, 728 cities' data includes both the number of 
criminal cases disposed and the number of cases satisfied by jail credit.



In [20]:
cities_with_data["pct_jail_per_case"] = (
    cities_with_data["jail_credit"] * 100.0 \
    / cities_with_data['criminal_case_disposed']
).round(1)

In [21]:
cities_with_data["rank"] = cities_with_data["pct_jail_per_case"].rank(ascending=False)

In [22]:
cities_with_data.head()

,jail_credit,criminal_case_disposed,population,pct_jail_per_case,rank
city,,,,,
Abernathy,48,749,2750,6.4,331.0
Abilene,13271,55740,120958,23.8,27.0
Addison,2185,27775,15457,7.9,281.5
Alamo,316,12251,19224,2.6,518.0
Alamo Heights,410,16077,7806,2.6,518.0


## "In fact, traffic and other fine-only offenses result in jail at a higher rate in Port Arthur than most other cities in the state, according to a BuzzFeed News analysis of Texas court data."

In [23]:
print("""
Of the {0} courts that reported both case-disposed and jail-credit data 
for the 2013-2015 fiscal years, Port Arthur had approximately the 
{1:.0f}th highest rate of cases resulting in jail time to pay off fines.
""".format(
        len(cities_with_data),
        cities_with_data.ix["Port Arthur"]["rank"]
    ))


Of the 728 courts that reported both case-disposed and jail-credit data 
for the 2013-2015 fiscal years, Port Arthur had approximately the 
10th highest rate of cases resulting in jail time to pay off fines.



### The table below shows the municipal courts that rank in the top 50 regarding the percentage of cases that result in jail time out of the total cases disposed.

* You'll notice that some courts reported more (a) cases where fines were resolved through jail credit than (b) total cases disposed. This results in a `pct_jail_per_case` that is higher than 100 percent. When asked about this, the Texas Office of Court Administration offered two possible explanations:

    * Total cases disposed counts the number of cases where a final judgement has been entered. So the jail credit counts could include cases that were disposed of months — or even years — ago but were only finally closed during FY 2013-15. 
    * Some high jail-credit counts could stem from data-entry or data-processing errors.

* More detail about how municipal courts are supposed to report their data is available [here](http://www.txcourts.gov/media/8263/Muni-Report-Instructions-2011-9_19_13.pdf).

In [24]:
cities_with_data[cities_with_data["rank"] <= 50].sort_values("rank", ascending=True)

,jail_credit,criminal_case_disposed,population,pct_jail_per_case,rank
city,,,,,
Naples,19550,1402,1363,1394.4,1.0
Edgecliff Village,997,379,2953,263.1,2.0
San Saba,331,430,2783,77.0,3.0
Pecos,610,1283,9213,47.5,4.0
Fort Stockton,221,583,8482,37.9,5.0
Waco,34723,92296,130194,37.6,6.0
West Orange,1156,3189,3455,36.2,7.0
Lamesa,1040,2968,9440,35.0,8.0
Breckenridge,513,1516,5545,33.8,9.0


---

---

---